In [1]:
import nltk

# conda install -c conda-forge wordcloud

from nltk.corpus import stopwords

#importing the required libraries
#!pip install wordCloud
# Libraries for data loading, data manipulation and data visulisation
import numpy as np                     
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
# Customise our plotting settings
sns.set_style('whitegrid')

#Libraries for data cleaning and preprocessing
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer
from sklearn.utils import resample
import string
import re
import pickle
import nltk
import string
stop = stopwords.words('english')

#Libraries for data preparation and model building
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC

from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score # Classification report

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression

#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine

In [2]:
#load the training and test data set
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
#checking the shape of the training dataframe
train.shape
print("The training dataset has {0} rows and {1} columns".format(train.shape[0], train.shape[1]))

The training dataset has 15819 rows and 3 columns


In [5]:
train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [7]:
#checking for unique values 
train['sentiment'].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [8]:
df_dist = train.groupby('sentiment').count()['message'].reset_index().sort_values(by='message', ascending=False)
df_dist.style.background_gradient(cmap='Purples')

,sentiment,message
2,1,8530
3,2,3640
1,0,2353
0,-1,1296


Well it looks like we have 4 unique values in the label.

Based on the description of the data, here is what each value stands for:

    1 Pro: the tweet supports the belief of man-made climate change
    2 News: the tweet links to factual news about climate change
    0 Neutral: the tweet neither supports nor refutes the belief of man-made climate change
    -1 Anti: the tweet does not believe in man-made climate change

In [10]:
df_train=train.copy()


In [11]:
#taking a colser look on the message column
df_train['message'] = df_train.message.str.lower()
df_train['message']

0        polyscimajor epa chief doesn't think carbon di...
1        it's not like we lack evidence of anthropogeni...
2        rt @rawstory: researchers say we have three ye...
3        #todayinmaker# wired : 2016 was a pivotal year...
4        rt @soynoviodetodas: it's 2016, and a racist, ...
                               ...                        
15814    rt @ezlusztig: they took down the material on ...
15815    rt @washingtonpost: how climate change could b...
15816    notiven: rt: nytimesworld :what does trump act...
15817    rt @sara8smiles: hey liberals the climate chan...
15818    rt @chet_cannon: .@kurteichenwald's 'climate c...
Name: message, Length: 15819, dtype: object

In [12]:
#creating a new dataframe for the features
df = pd.DataFrame(df_train['message'])

In [13]:
# Remove Punctuation
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
#Data preprocessing
#function that handles the removal punctuations from the tweets
def remove_punct(text):
    """
    the function remove_punction, it takes in a text as input and loops through
    the text, if a character is not in string.punctuation then it adds the character
    as a string to the text variable
    
    """
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [15]:
#removes all websites and replaces them with the text 'web-url'
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
df['message_punct'] = df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)


In [16]:
# Data cleaning 
def clean_data(texts):
    
    """
    clean_data(text), the function further cleans the data using (re)
    by removing extract white spaces and non text characters
    
    """
    words = list()
    for text in texts.split():
        # remove non text character from start and end of string
        text = re.sub(r'(^\W+|\W+$)','',text)
#       #remove multiple white spaces
        text = re.sub(r'\s+','',text)
#       #remove non text characters and emojis between texts
        text = re.sub(r'\W+',r'',text)
#       #remove white space at the end of strings
        text = re.sub(r'\s+$',r'',text)
#       #Remove unwanted symbols
        text = re.sub(r'[#,@,$_,?*//""]',r'',text)
        words.append(text.lower())
            
        text = [i for i in words if len(i) >= 2]
        meaningful_words = [w for w in text if not w in stop]

    return " ".join(meaningful_words)

In [17]:
#applying the clean_data function
df['tweets'] = df['message_punct'].apply(clean_data)

In [18]:
df.head()


,message,message_punct,tweets
0,polyscimajor epa chief doesn't think carbon di...,polyscimajor epa chief doesn't think carbon di...,polyscimajor epa chief doesnt think carbon dio...
1,it's not like we lack evidence of anthropogeni...,it's not like we lack evidence of anthropogeni...,like lack evidence anthropogenic global warming
2,rt @rawstory: researchers say we have three ye...,rt @rawstory: researchers say we have three ye...,rt rawstory researchers say three years act cl...
3,#todayinmaker# wired : 2016 was a pivotal year...,#todayinmaker# wired : 2016 was a pivotal year...,todayinmaker wired 2016 pivotal year war clima...
4,"rt @soynoviodetodas: it's 2016, and a racist, ...","rt @soynoviodetodas: it's 2016, and a racist, ...",rt soynoviodetodas 2016 racist sexist climate ...


In [19]:
#applying tokenization to the data set
tokeniser = TreebankWordTokenizer()
df['tokens'] = df['tweets'].apply(tokeniser.tokenize)

In [20]:
#applying Lammetization
lemmatizer = WordNetLemmatizer()

In [21]:
df.head()

,message,message_punct,tweets,tokens
0,polyscimajor epa chief doesn't think carbon di...,polyscimajor epa chief doesn't think carbon di...,polyscimajor epa chief doesnt think carbon dio...,"[polyscimajor, epa, chief, doesnt, think, carb..."
1,it's not like we lack evidence of anthropogeni...,it's not like we lack evidence of anthropogeni...,like lack evidence anthropogenic global warming,"[like, lack, evidence, anthropogenic, global, ..."
2,rt @rawstory: researchers say we have three ye...,rt @rawstory: researchers say we have three ye...,rt rawstory researchers say three years act cl...,"[rt, rawstory, researchers, say, three, years,..."
3,#todayinmaker# wired : 2016 was a pivotal year...,#todayinmaker# wired : 2016 was a pivotal year...,todayinmaker wired 2016 pivotal year war clima...,"[todayinmaker, wired, 2016, pivotal, year, war..."
4,"rt @soynoviodetodas: it's 2016, and a racist, ...","rt @soynoviodetodas: it's 2016, and a racist, ...",rt soynoviodetodas 2016 racist sexist climate ...,"[rt, soynoviodetodas, 2016, racist, sexist, cl..."


In [22]:
#function that handles the process of lemmatization
def extract_lemma(words, lemmatizer):
    return ' '.join([lemmatizer.lemmatize(word) for word in words])   

In [23]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]  ##Notice the use of text.

In [24]:
#calling extract_lemma function on the tokens column
df['lemma'] = df['tokens'].apply(extract_lemma, args=(lemmatizer, ))

In [25]:
df.head()

,message,message_punct,tweets,tokens,lemma
0,polyscimajor epa chief doesn't think carbon di...,polyscimajor epa chief doesn't think carbon di...,polyscimajor epa chief doesnt think carbon dio...,"[polyscimajor, epa, chief, doesnt, think, carb...",polyscimajor epa chief doesnt think carbon dio...
1,it's not like we lack evidence of anthropogeni...,it's not like we lack evidence of anthropogeni...,like lack evidence anthropogenic global warming,"[like, lack, evidence, anthropogenic, global, ...",like lack evidence anthropogenic global warming
2,rt @rawstory: researchers say we have three ye...,rt @rawstory: researchers say we have three ye...,rt rawstory researchers say three years act cl...,"[rt, rawstory, researchers, say, three, years,...",rt rawstory researcher say three year act clim...
3,#todayinmaker# wired : 2016 was a pivotal year...,#todayinmaker# wired : 2016 was a pivotal year...,todayinmaker wired 2016 pivotal year war clima...,"[todayinmaker, wired, 2016, pivotal, year, war...",todayinmaker wired 2016 pivotal year war clima...
4,"rt @soynoviodetodas: it's 2016, and a racist, ...","rt @soynoviodetodas: it's 2016, and a racist, ...",rt soynoviodetodas 2016 racist sexist climate ...,"[rt, soynoviodetodas, 2016, racist, sexist, cl...",rt soynoviodetodas 2016 racist sexist climate ...


In [26]:
#using countVectorizer
vectorizer = CountVectorizer(lowercase=True, stop_words='english', analyzer='word', ngram_range=(1, 1))

In [27]:
X_count = vectorizer.fit_transform(df['lemma'].values.astype(str))

In [28]:
X_count

<15819x23084 sparse matrix of type '<class 'numpy.int64'>'
	with 179384 stored elements in Compressed Sparse Row format>

In [31]:
X_count.shape

(15819, 23084)

In [32]:
# Determine our Label
y = df_train['sentiment']

In [34]:
X = X_count.toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [35]:
# concatenate both features and labels before resampling
data = np.concatenate([X, y[:,np.newaxis]], axis=1)
data

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_10376/1244224774.py:2: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  data = np.concatenate([X, y[:,np.newaxis]], axis=1)


array([[ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  2],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0, -1],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int64)

In [36]:
#creating a test dataframe
test_df = pd.DataFrame(test['message'])

In [37]:
test_df.head()

,message
0,Europe will now be looking to China to make su...
1,Combine this with the polling of staffers re c...
2,"The scary, unimpeachable evidence that climate..."
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...
4,RT @FakeWillMoore: 'Female orgasms cause globa...


In [38]:
#removing http and replacing it with url pattern
test_df['message_punct'] = test_df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [39]:
#apply the clean_data function
test_df['tweets'] = test_df['message_punct'].apply(clean_data)

In [40]:
#applying tokenizer
test_df['tokens'] = test_df['tweets'].apply(tokeniser.tokenize)

In [41]:
#applying the extract_lemma function
test_df['lemma'] = test_df['tokens'].apply(extract_lemma, args=(lemmatizer, ))

In [42]:
#transforming the data using vectorizer
test_count = vectorizer.transform(test_df['lemma'].values.astype(str))

In [43]:
#selecting the feature
x_test = test_count.toarray()

In [44]:
#the shape of the feature
x_test.shape

(10546, 23084)

In [45]:
# Split Data (into Training & Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [46]:
# svc_linear = LinearSVC(random_state=0, tol=1e-5)
svc = SVC()
f_clf = RandomForestClassifier(random_state=0)
nv_clf = MultinomialNB()
lg_clf = LogisticRegression(random_state = 0)
passive = PassiveAggressiveClassifier()

In [47]:
#fitting the models 
svc.fit(X_train, y_train)
f_clf.fit(X_train, y_train)
nv_clf.fit(X_train, y_train)
lg_clf.fit(X_train, y_train)
passive.fit(X_train, y_train)

KeyboardInterrupt: 